# Web Server Logs

#### Analysis using pyspark

**Name: Darren Foley**
**Date: 2021-10-05**

In [1]:
!pip install pandas
!pip install findspark

In [15]:
access_logs = "data/access_log_sample.txt"

In [24]:
import pandas as pd
import findspark
import re
import os

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, col, regexp_extract
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read("aws.cfg")

AWS_ACCESS_KEY_M = config.get('MASTER', 'AWS_ACCESS_KEY')
AWS_SECRET_KEY_M = config.get('MASTER', 'AWS_SECRET_KEY')

AWS_ACCESS_KEY_K = config.get('KINESIS', 'AWS_ACCESS_KEY')
AWS_SECRET_KEY_K = config.get('KINESIS', 'AWS_SECRET_KEY')

AWS_ACCESS_KEY_F = config.get('FIREHOSE', 'AWS_ACCESS_KEY')
AWS_SECRET_KEY_F = config.get('FIREHOSE', 'AWS_SECRET_KEY')

#print(AWS_ACCESS_KEY_M)
#print(AWS_SECRET_KEY_K)

In [11]:
findspark.init()

In [12]:
spark = SparkSession \
    .builder \
    .appName("WebLogsAnalysis") \
    .getOrCreate()

In [27]:
df = spark.read \
    .option("delimiter", "\n") \
    .option("header", False) \
    .csv(access_logs)

df = df.withColumnRenamed("_c0", "raw_logs")

In [28]:
df.show().toPandas()

+--------------------+
|            raw_logs|
+--------------------+
|54.36.149.41 - - ...|
|31.56.96.51 - - [...|
|31.56.96.51 - - [...|
|40.77.167.129 - -...|
|91.99.72.15 - - [...|
|40.77.167.129 - -...|
|40.77.167.129 - -...|
|40.77.167.129 - -...|
|66.249.66.194 - -...|
|40.77.167.129 - -...|
|207.46.13.136 - -...|
|40.77.167.129 - -...|
|178.253.33.51 - -...|
|40.77.167.129 - -...|
|91.99.72.15 - - [...|
|40.77.167.129 - -...|
|207.46.13.136 - -...|
|40.77.167.129 - -...|
|40.77.167.129 - -...|
|66.249.66.194 - -...|
+--------------------+
only showing top 20 rows



AttributeError: 'NoneType' object has no attribute 'toPandas'

### Convert raw logs into dataframe using regex

### Schema defined based on regex groups

In [37]:
combined_regex = '^(\S+) (\S+) (\S+) \[(\w{1,2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}\s[+\-]\d{4})\] "(\S+)\s?(\S+)?\s?(\S+)?" (\d{3}|-) (\d+|-)\s?"?([^"]*)"?\s?"?([^"]*)?"\s?"?([^"]*)?"?$'


log_schema = StructType([StructField("client_ip",StringType(),True), \
                         StructField("rfc931",DoubleType(),True), \
                         StructField("username",StringType(),True), \
                         StructField("datetime",DoubleType(),True), \
                         StructField("http_method",StringType(),True), \
                         StructField("http_url",DoubleType(),True), \
                         StructField("http_protocol",LongType(),True), \
                         StructField("http_status_code",StringType(),True), \
                         StructField("bytes_sent",StringType(),True), \
                         StructField("referrer_url",LongType(),True), \
                         StructField("user_agent",StringType(), True), \
                         StructField("cookies", StringType(), True)])

df_parsed = df \
            .withColumn("client_ip", regexp_extract(col("raw_logs"), combined_regex, 1)) \
            .withColumn("rfc931", regexp_extract(col("raw_logs"), combined_regex, 2)) \
            .withColumn("username", regexp_extract(col("raw_logs"), combined_regex, 3)) \
            .withColumn("datetime", regexp_extract(col("raw_logs"), combined_regex, 4)) \
            .withColumn("http_method", regexp_extract(col("raw_logs"), combined_regex, 5)) \
            .withColumn("http_url", regexp_extract(col("raw_logs"), combined_regex, 6)) \
            .withColumn("http_protocol", regexp_extract(col("raw_logs"), combined_regex, 7)) \
            .withColumn("http_status_code", regexp_extract(col("raw_logs"), combined_regex, 8)) \
            .withColumn("bytes_sent", regexp_extract(col("raw_logs"), combined_regex, 9)) \
            .withColumn("referrer_url", regexp_extract(col("raw_logs"), combined_regex, 10)) \
            .withColumn("user_agent", regexp_extract(col("raw_logs"), combined_regex, 11)) \
            .withColumn("cookies", regexp_extract(col("raw_logs"), combined_regex, 12))

df_parsed.limit(5).toPandas()

,raw_logs,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies
0,54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] ...,54.36.149.41,-,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,HTTP/1.1,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,-
1,"31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] ""...",31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-
2,"31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] ""...",31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-
3,40.77.167.129 - - [22/Jan/2019:03:56:17 +0330]...,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-
4,"91.99.72.15 - - [22/Jan/2019:03:56:17 +0330] ""...",91.99.72.15,-,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,-


### Pyspark Streaming job

### 